In [5]:
#импортирование нужных библиотек
import pandas as pd
from datetime import datetime 
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [6]:
# Файл с информацией о клиентах
first_file = pd.read_csv('resto-asap2018\CASC_Constant.csv') 
print(first_file.head())
print(first_file.info())

   CustomerId ActivationDate   Age     Sex  SubscribedEmail  SubscribedPush
0     2728183     2015-01-01  21.0  Female            False            True
1     2728198     2015-01-01  21.0  Female             True            True
2     2728306     2015-01-01  21.0  Female             True            True
3     2728178     2015-01-01  22.0    Male             True            True
4     2728322     2015-01-01  22.0    Male             True            True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   ActivationDate   10000 non-null  object 
 2   Age              9953 non-null   float64
 3   Sex              8874 non-null   object 
 4   SubscribedEmail  10000 non-null  bool   
 5   SubscribedPush   10000 non-null  bool   
dtypes: bool(2), float64(1), int64(1), object(2)
memory usage: 332.2+

In [7]:
# Файл с информацией о походах клиентов в рестораны
second_file = pd.read_csv('resto-asap2018\casc-resto.csv', sep=';', low_memory=False, decimal=',')
print(second_file.head())
print(second_file.info())

   CustomerID  Restaurant      RKDate                    RegionName  \
0     2898197         391  2017-07-16  Москва и Московская область    
1     2903215          43  2015-04-07  Москва и Московская область    
2     2748887          43  2015-05-22  Москва и Московская область    
3     2862077          46  2015-03-05  Москва и Московская область    
4     2862077          46  2015-03-11  Москва и Московская область    

   BrandsNames DishCategoryName  Quantity  SummBasic  SummAfterPointsUsage  
0  TGI FRIDAYS      NON ALCOHOL         2        2.0                   2.0  
1     IL Патио      NON ALCOHOL         1        5.0                   5.0  
2     IL Патио      NON ALCOHOL         1        5.0                   5.0  
3     IL Патио      NON ALCOHOL         1        5.0                   5.0  
4     IL Патио      NON ALCOHOL         1        5.0                   5.0  


In [8]:
# Файл с информацией о походах клиентов в рестораны (сжатая версия, далее не используется)
third_file = pd.read_csv('resto-asap2018\casc-resto-small.csv')
#print(third_file.head())
#print(third_file.info())

In [9]:
# Определить, совершал ли клиент визит в период между 2017-07-01 и 2017-12-31
start_date = datetime(2017,7,1)
final_date = datetime(2017,12,31)
second_file['RKDate'] = pd.to_datetime(second_file['RKDate'])
grouped = second_file.groupby(by='CustomerID',sort=False)
second_file['BetweenTwoDates']=grouped['RKDate'].apply(lambda x: (x>=start_date)&(x<=final_date)) 
second_file = second_file.reset_index()
print(second_file.head())

   index  CustomerID  Restaurant     RKDate                    RegionName  \
0      0     2898197         391 2017-07-16  Москва и Московская область    
1      1     2903215          43 2015-04-07  Москва и Московская область    
2      2     2748887          43 2015-05-22  Москва и Московская область    
3      3     2862077          46 2015-03-05  Москва и Московская область    
4      4     2862077          46 2015-03-11  Москва и Московская область    

   BrandsNames DishCategoryName  Quantity  SummBasic  SummAfterPointsUsage  \
0  TGI FRIDAYS      NON ALCOHOL         2        2.0                   2.0   
1     IL Патио      NON ALCOHOL         1        5.0                   5.0   
2     IL Патио      NON ALCOHOL         1        5.0                   5.0   
3     IL Патио      NON ALCOHOL         1        5.0                   5.0   
4     IL Патио      NON ALCOHOL         1        5.0                   5.0   

   BetweenTwoDates  
0             True  
1            False  
2    

In [69]:
# Расчёт Recency
def recency(dataframe):
    get_dates = dataframe[dataframe.RKDate<start_date].RKDate
    if get_dates.size==0:
        return 0
    else:
        max_dates = get_dates.max()
        return (start_date - max_dates).days

In [70]:
# расчёт frequency
def frequency(dataframe):
    first_visit = dataframe['RKDate'].min()
    if first_visit >= start_date:
        return 0
    days_count = (start_date - first_visit).days
    month_count = math.ceil(days_count / 30)
    visit_count = dataframe.RKDate.size    
    return visit_count/month_count

In [71]:
# расчёт monetary_value
def monetary_value(dataframe):
    correct_dates = dataframe[dataframe.RKDate < start_date]
    unique_dates = correct_dates.RKDate.unique()
    if unique_dates.size == 0:
        return 0
    else:
        point_usage_sum = correct_dates.SummAfterPointsUsage.sum()
        return point_usage_sum / unique_dates.size

In [72]:
# сумма скидок при всех посещениях
def point_usage(dataframe):    
    get_dates = dataframe[dataframe.RKDate<start_date]
    basic_sum = get_dates.SummBasic.sum()
    point_usage_sum = get_dates.SummAfterPointsUsage.sum()
    return basic_sum - point_usage_sum

In [73]:
# формирование таблицы
def all_analytics(dataframe):
    coloumns = dict({
        "Recency": recency(dataframe),
        "Frequency": frequency(dataframe),
        "Monetary value": monetary_value(dataframe),
        "Point usage": point_usage(dataframe)
    })
    return pd.Series(coloumns)

In [74]:
# формирование DataFrame с нужными для модели данными
customers_analytics = second_file.groupby(by = "CustomerID", sort=False).apply(all_analytics)
print(customers_analytics.head())
print(customers_analytics.info())

            Recency  Frequency  Monetary value  Point usage
CustomerID                                                 
2898197        64.0   7.821429     2695.222222       4754.0
2903215       367.0   1.714286     1697.875000       1057.0
2748887        23.0   7.433333     1241.320000       7020.0
2862077       761.0   5.000000      565.000000       1123.0
2801997       458.0   3.827586      580.033333       1347.5
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2898197 to 2744732
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Recency         10000 non-null  float64
 1   Frequency       10000 non-null  float64
 2   Monetary value  10000 non-null  float64
 3   Point usage     10000 non-null  float64
dtypes: float64(4)
memory usage: 390.6 KB
None


In [75]:
# установка индексов
customers_analytics = customers_analytics.reset_index()
print(customers_analytics.head())

   CustomerID  Recency  Frequency  Monetary value  Point usage
0     2898197     64.0   7.821429     2695.222222       4754.0
1     2903215    367.0   1.714286     1697.875000       1057.0
2     2748887     23.0   7.433333     1241.320000       7020.0
3     2862077    761.0   5.000000      565.000000       1123.0
4     2801997    458.0   3.827586      580.033333       1347.5


In [76]:
#изъятие информации о возрасте и поле клиентов
customers_info = first_file[["CustomerId","Age","Sex"]]
print(customers_info.head())

   CustomerId   Age     Sex
0     2728183  21.0  Female
1     2728198  21.0  Female
2     2728306  21.0  Female
3     2728178  22.0    Male
4     2728322  22.0    Male


In [77]:
# соединение двух таблиц
data_for_model = pd.merge(left = customers_analytics, right = customers_info, left_on="CustomerID", right_on="CustomerId")
data_for_model.drop("CustomerId", axis=1, inplace = True)
mean_Age = data_for_model['Age'].mean()
data_for_model['Age'] = data_for_model.Age.fillna(mean_Age)
data_for_model.Sex.value_counts(dropna=False)

Female    5141
Male      3733
NaN       1126
Name: Sex, dtype: int64

In [78]:
# декодинг пола клиента 
def sex_to_numbers(sex):
    if sex == 'Male':
        return 1
    elif sex == 'Female':
        return 2
    else:
        return 0

In [118]:
# формирование обучающей и тестовой выборки
data_for_model.Sex = data_for_model.Sex.apply(sex_to_numbers)
x = data_for_model[['Recency','Frequency','Monetary value','Point usage','Age','Sex']].to_numpy()
unique_people = second_file.groupby(by='CustomerID').BetweenTwoDates.max()
y = unique_people.values
print(x.size)
print(y.size)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

60000
10000


In [119]:
# обучение
regression = LogisticRegression()
regression.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [123]:
# вероятность посещения на тестовой выборке
y_predicted = regression.predict(x_test)
print(confusion_matrix(y_test,y_predicted))
print(regression.score(x_test, y_test))

[[1069   10]
 [ 909   12]]
0.5405


In [121]:
# подсчёт precision и recall
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

       False       0.54      0.99      0.70      1079
        True       0.55      0.01      0.03       921

    accuracy                           0.54      2000
   macro avg       0.54      0.50      0.36      2000
weighted avg       0.54      0.54      0.39      2000

